In [0]:
%pip install labelme

Python interpreter will be restarted.
Requirement already satisfied: labelme in /local_disk0/.ephemeral_nfs/envs/pythonEnv-ff5e88f7-8225-4ab4-aca7-9a0372d27ec4/lib/python3.8/site-packages (4.5.13)
Requirement already satisfied: PyYAML in /local_disk0/.ephemeral_nfs/envs/pythonEnv-ff5e88f7-8225-4ab4-aca7-9a0372d27ec4/lib/python3.8/site-packages (from labelme) (5.4.1)
Requirement already satisfied: qtpy in /local_disk0/.ephemeral_nfs/envs/pythonEnv-ff5e88f7-8225-4ab4-aca7-9a0372d27ec4/lib/python3.8/site-packages (from labelme) (1.11.2)
Requirement already satisfied: matplotlib<3.3 in /databricks/python3/lib/python3.8/site-packages (from labelme) (3.2.2)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.8/site-packages (from labelme) (1.19.2)
Requirement already satisfied: termcolor in /local_disk0/.ephemeral_nfs/envs/pythonEnv-ff5e88f7-8225-4ab4-aca7-9a0372d27ec4/lib/python3.8/site-packages (from labelme) (1.1.0)
Requirement already satisfied: Pillow>=2.8 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-ff5e88f7-8225-4ab4-aca7-9a0372d27ec4/lib/python3.8/site-packages (from labelme) (8.3.2)
Requirement already satisfied: PyQt5!=5.15.3,!=5.15.4 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-ff5e88f7-8225-4ab4-aca7-9a0372d27ec4/lib/python3.8/site-packages (from labelme) (5.15.2)
Requirement already satisfied: imgviz>=0.11 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-ff5e88f7-8225-4ab4-aca7-9a0372d27ec4/lib/python3.8/site-packages (from labelme) (1.4.0)
Requirement already satisfied: cycler>=0.10 in /databricks/python3/lib/python3.8/site-packages (from matplotlib<3.3->labelme) (0.10.0)
Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /databricks/python3/lib/python3.8/site-packages (from matplotlib<3.3->labelme) (2.4.7)
Requirement already satisfied: python-dateutil>=2.1 in /databricks/python3/lib/python3.8/site-packages (from matplotlib<3.3->labelme) (2.8.1)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/python3/lib/python3.8/site-packages (from matplotlib<3.3->labelme) (1.3.0)
Requirement already satisfied: PyQt5-sip<13,>=12.8 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-ff5e88f7-8225-4ab4-aca7-9a0372d27ec4/lib/python3.8/site-packages (from PyQt5!=5.15.3,!=5.15.4->labelme) (12.9.0)
Requirement already satisfied: six in /databricks/python3/lib/python3.8/site-packages (from cycler>=0.10->matplotlib<3.3->labelme) (1.15.0)
WARNING: You are using pip version 20.2.4; however, version 21.2.4 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-ff5e88f7-8225-4ab4-aca7-9a0372d27ec4/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
# import the needed libraries
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from pathlib import Path
import labelme

In [0]:
# Lets have a look into the labels we are working with, and the scores for the three sub use cases
label_scores = pd.read_csv('/dbfs/mnt/sdscdata/data/raw/label_scores.csv', sep=';')
labels = list(label_scores.labels.values)

train_jpg = []
train_json = []
for file in dbutils.fs.ls("/mnt/sdscdata/data/raw/train"):
  if(file.name[-4:] == 'jpeg'):
    train_jpg.append(file.name)
  if(file.name[-4:] == 'json'):
    train_json.append(file.name)

In [0]:
def display_image(index):
  #path to the image
  jpeg_path = Path('/dbfs/mnt/sdscdata/data/raw/train/', train_jpg[index])
  json_path = Path('/dbfs/mnt/sdscdata/data/raw/train/', train_json[index])
  
  #loading the image
  img = mpimg.imread(jpeg_path)
  img_mask = mask_from_json(json_path)
  collision_mask = get_collision_mask(img_mask)
  
  display_all_layers(img_mask, img, collision_mask)
  

In [0]:
def display_all_layers(mask, img, collision_mask):
  mask = mask.astype(int)
  # creating multiple subplots
  fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(20,20), gridspec_kw = {'wspace':0.01, 'hspace':0})

  # title for the entire figure
  fig.suptitle('Every layers of the mask', fontsize=20)
  
  titles = labels.copy()
  titles.append('Train image')
  titles.append('Collision mask')
  

  # filling subplots with pictures
  for i, ax in enumerate(fig.axes):
    if i < mask.shape[2]:
      ax.imshow(mask[:,:,i], vmin=0, vmax=1)
    # removing axes of the subplots
    if(i == len(fig.axes)-2):
       ax.imshow(img)
    if(i == len(fig.axes)-1):
       ax.imshow(collision_mask.astype(int), vmin=0, vmax=1)
    ax.set_title(titles[i])
    ax.axis('off')

In [0]:
def get_collision_mask(mask):
  mask = mask.astype(int)
  intersection = np.sum(mask, axis=-1)
  #remove uncollided area
  intersection[intersection == 1] = 0
  #put all collision area to 1
  intersection[intersection >= 2] = 1
  
  return intersection

In [0]:
def collision_ratio(mask):
  mask.astype(int)
  intersection = np.sum(mask, axis=-1)
  #remove uncollided area
  intersection[intersection == 1] = 0
  #put all collision area to 1
  intersection[intersection >= 2] = 1
  
  return float(np.sum(intersection))/(512*512)

In [0]:
def mask_from_json(json_path):
  with open(json_path, "r") as f:
      dj = json.load(f)
  
  #18 layers for the 18 labels
  masks = np.zeros((512,512,18), dtype=bool)
  
  for i in range(len(dj['shapes'])):
    index = labels.index(dj['shapes'][i]['label'])
    #print(np.unique(labelme.utils.shape_to_mask((dj['imageHeight'],dj['imageWidth']), dj['shapes'][i]['points'], shape_type=None,line_width=1, point_size=1))*i)
    #masks.append((labelme.utils.shape_to_mask((dj['imageHeight'],dj['imageWidth']), dj['shapes'][i]['points'], shape_type=None,line_width=1, point_size=1)*i).astype(np.int))
    label_mask = labelme.utils.shape_to_mask((dj['imageHeight'],dj['imageWidth']), dj['shapes'][i]['points'], shape_type=dj['shapes'][i]['shape_type'],line_width=1, point_size=1)
    masks[:,:,index] = np.logical_or(masks[:,:,index], label_mask)
  #masks_stacked = np.stack(masks, axis=-1)
  return masks.astype(np.int)

In [0]:
def get_label_ratio(mask, label):
    #get correct mask
    index = labels.index(label)
    mask = mask[:,:,index].astype(int)

    numerator = np.sum(mask)
    denominator = mask.shape[0]*mask.shape[1]
    
    return float(numerator)/denominator